In [ ]:
from typing import Optional, List
from pydantic import Field
from dslmodel import DSLModel, init_instant
init_instant()



In [1]:
class Message(DSLModel):
    content: str = Field(..., description="The content of the message")
    role: str = Field(..., description="The role of the sender (e.g., 'assistant', 'user', 'system')")
    name: Optional[str] = Field(default=None, description="The name of the sender (if applicable)")
    

class Completion(DSLModel):
    messages: List[Message] = Field(..., description="A list of messages comprising the completion")

In [2]:
import dspy
from dslmodel import DSLModel, init_instant
from pydantic import Field
from typing import List, Optional, Dict
import openai

# Initialize the DSLModel environments
init_instant()

# Set OpenAI API key
openai.api_key = "your-api-key"


# Define the Message model
class Message(DSLModel):
    content: str = Field(..., description="The content of the message")
    role: str = Field(..., description="The role of the sender (e.g., 'assistant', 'user', 'system')")
    name: Optional[str] = Field(default=None, description="The name of the sender (if applicable)")


# Define the ToolFunction model
class ToolFunction(DSLModel):
    name: str = Field(..., description="The name of the tool (function)")
    description: str = Field(..., description="The description of the tool")
    parameters: Dict[str, Dict] = Field(..., description="The parameters required by the tool")


# Define the Tool model
class Tool(DSLModel):
    type: str = Field("function", description="The type of the tool")
    function: ToolFunction = Field(..., description="The function details for the tool")


# Define the tool for getting the delivery date
tool_data = {
    "type": "function",
    "function": {
        "name": "get_delivery_date",
        "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date.",
        "parameters": {
            "order_id": {
                "type": "string",
                "description": "The customer's order ID."
            }
        }
    }
}

tool = Tool(**tool_data)

# Define the system and user messages
messages_data = [
    {"role": "system",
     "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, can you tell me the delivery date for my order?"}
]

# Create Message instances using from_prompt
messages = [Message(**msg) for msg in messages_data]

print(tool, messages)

In [4]:
example = """from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed
"""

# Our DSLModel version of the above
from typing import TypeVar, Type
T = TypeVar("T", bound=DSLModel)


class CalendarEvent(DSLModel):
    name: str
    date: str
    participants: list[str]


class Completions(DSLModel):
    @classmethod
    def parse(cls, messages: list[Message], response_format: Type[T]) -> T:
        return response_format.from_prompt(messages)
    

completion = Completions.parse(example, CalendarEvent)
print(completion)

In [6]:
completion = CalendarEvent.from_prompt(example)
print(completion)

In [9]:
class GetDeliveryDate(DSLModel):
    order_id: str
    
msgs = """messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order #12345?"}) 
""" 

date = GetDeliveryDate.from_prompt(msgs)

In [14]:
print(date.to_toml())